# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
from azureml.core.compute_target import ComputeTargetException
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import azureml.core
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory

In [2]:
print(azureml.core.VERSION)

1.48.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'ml-exp'
project_folder = "."

exp=Experiment(ws, experiment_name)



amlcompute_cluster_name = "mycluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
    max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

In [4]:
dataset = Dataset.get_by_name(ws, name='heart_disease')
data = dataset.to_pandas_dataframe()
data.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
import logging
#Put your automl settings here
automl_settings = {"experiment_timeout_minutes": 30,
                   "max_concurrent_iterations": 4,
                   "primary_metric" : 'accuracy'}

#Put your automl config here
automl_config = AutoMLConfig(task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",   
                             path = project_folder,
                             compute_target=compute_target,
                             enable_early_stopping= True,
                             featurization= 'off',
                             enable_onnx_compatible_models=True,
                             debug_log = "automl_errors.log",
                             **automl_settings)

In [8]:
# TODO: Submit your experiment
remote_run = exp.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
ml-exp,AutoML_a5a4edd9-1f2f-42fd-9cab-60df0b1cd4a9,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
ml-exp,AutoML_a5a4edd9-1f2f-42fd-9cab-60df0b1cd4a9,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  In order to accurately evaluate the model(s) trained by AutoML, we leverage a dataset that the model is not trained on. Hence, if the user doesn't provide an explicit validation dataset, a part of the training dataset is used to achieve this. For smaller datasets (fewer than 20,000 samples), cross-validation is leveraged, else a single hold-out set is split from the training data to serve as the validation dataset. Hence, for your input data we leverage cross-validation with 10 folds, if the number of training samples are fewer than 1000, and 3 folds in all other cases.
              Learn more about cross validation: https://aka.ms/AutomatedMLCrossValidation
DETAILS:      
+------------------------------+
|Number of folds               |
+=============

{'runId': 'AutoML_a5a4edd9-1f2f-42fd-9cab-60df0b1cd4a9',
 'target': 'mycluster',
 'status': 'Completed',
 'startTimeUtc': '2023-02-02T13:01:47.46969Z',
 'endTimeUtc': '2023-02-02T13:14:20.283096Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'mycluster',
  'AMLSettingsJsonString': '{"path":null,"name":"ml-exp","subscription_id":"3e42d11f-d64d-4173-af9b-12ecaa1030b3","resource_group":"aml-quickstarts-224429","workspace_name":"quick-starts-ws-224429","region":"westus2","compute_target":"mycluster","spark_service":null,"a

KeyError: 'log_files'

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [18]:
#save best model
import joblib
best_run, fitted_model = remote_run.get_output()
print('Best run:',best_run)
print('Best fitted model:', fitted_model)

joblib.dump (fitted_model, 'automlmodel.joblib')

Best run: Run(Experiment: ml-exp,
Id: AutoML_a5a4edd9-1f2f-42fd-9cab-60df0b1cd4a9_59,
Type: azureml.scriptrun,
Status: Completed)
Best fitted model: Pipeline(memory=None,
         steps=[('prefittedsoftvotingclassifier',
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('42', Pipeline(memory=None, steps=[('SparseNormalizer', Normalizer(copy=True, norm='l2')), ('XGBoostClassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=0.6, eta=0.2, gamma=0.1, max_depth=8, max_leaves=0, n_est...7708333333333335, reg_lambda=2.3958333333333335, subsample=0.6, tree_method='auto'))], verbose=False))], flatten_transform=None, weights=[0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.18181818181818182, 0.09090909090909091, 0.09090909090909091]))],
         verbose=False)


['automlmodel.joblib']

In [21]:
best_run.get_metrics()["accuracy"]

0.8897701149425288

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [22]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model
from azureml.train.automl import constants
from azureml.automl.core.shared import constants


#deploying the already registed model (best), creating an inference and --> model exposed as a WS-*.
#getting environment from best run
env = best_run.get_environment()

#registeting the model
#model = best_run.register_model(model_name='best_automodel_lr_automl.pkl')
model = remote_run.register_model(model_name=best_run.properties['model_name'])


best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')
best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'env.yml')
inference_config = InferenceConfig(entry_script='score.py', environment=env)

deployment_config = AciWebservice.deploy_configuration(
                                                       cpu_cores=1,
                                                       memory_gb=1,
                                                       description='predicting heart failure',
                                                       auth_enabled=True,
                                                       enable_app_insights= True,
                                                       collect_model_data = True)



#inference_config = InferenceConfig(entry_script='score.py',environment=env)
#deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
service = Model.deploy(ws, 'heart-failure-service', [model], inference_config, deployment_config, overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-02-02 13:51:53+00:00 Creating Container Registry if not exists..
2023-02-02 14:01:53+00:00 Registering the environment.
2023-02-02 14:01:54+00:00 Use the existing image.
2023-02-02 14:01:54+00:00 Submitting deployment to compute.

TODO: In the cell below, send a request to the web service you deployed to test it.

In [23]:
import requests
import json 
print('Score URI:',service.scoring_uri)
print('Swagger URL:',service.swagger_uri)



#it could have done also with and endpoint.py script as in the project 2, but for simplicity just make a qury with a payload and auth.
#testing with dummy values
import urllib.request
import json
import os
import ssl



data = {
    "data":
    [
        {
            'age': "40",
            'anaemia': "0",
            'creatinine_phosphokinase': "315",
            'diabetes': "1",
            'ejection_fraction': "20",
            'high_blood_pressure': "0",
            'platelets': "162000",
            'serum_creatinine': "2.7",
            'serum_sodium': "500",
            'sex': "0",
            'smoking': "0",
            'time': "10",
        },
    ],
}


body = str.encode(json.dumps(data))

headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ service.get_keys()[0])}

req = urllib.request.Request(service.scoring_uri, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))

Score URI: http://07ea6e02-211f-4254-ad41-cd38ea579dab.westus2.azurecontainer.io/score
Swagger URL: http://07ea6e02-211f-4254-ad41-cd38ea579dab.westus2.azurecontainer.io/swagger.json
b'"{\\"result\\": [1]}"'


TODO: In the cell below, print the logs of the web service and delete the service

In [24]:
print(service.get_logs())

2023-02-02T14:03:59,245059852+00:00 - rsyslog/run 
2023-02-02T14:03:59,256769300+00:00 - iot-server/run 
2023-02-02T14:03:59,258287220+00:00 - gunicorn/run 
2023-02-02T14:03:59,263406279+00:00 | gunicorn/run | 
2023-02-02T14:03:59,268881773+00:00 | gunicorn/run | ###############################################
2023-02-02T14:03:59,274115511+00:00 | gunicorn/run | AzureML Container Runtime Information
2023-02-02T14:03:59,279670289+00:00 | gunicorn/run | ###############################################
2023-02-02T14:03:59,284820343+00:00 | gunicorn/run | 
2023-02-02T14:03:59,291944233+00:00 | gunicorn/run | 
2023-02-02T14:03:59,319396086+00:00 | gunicorn/run | AzureML image information: openmpi3.1.2-ubuntu18.04, Materializaton Build:20230103.v4
2023-02-02T14:03:59,321426013+00:00 - nginx/run 
2023-02-02T14:03:59,322018904+00:00 | gunicorn/run | 
2023-02-02T14:03:59,331235810+00:00 | gunicorn/run | 
2023-02-02T14:03:59,334356436+00:00 | gunicorn/run | PATH environment variable: /azureml-env

In [25]:
service.delete()

In [29]:
service.state

'Deleting'

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
